<a href="https://colab.research.google.com/github/girija2204/DLTopics/blob/master/Haar_Cascade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import numpy as np
import datetime

In [ ]:
trained_face_data = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
trained_eye_data = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
def cut_face_from_image(img, coordinates):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.equalizeHist(img_gray)
    img_gray = img_gray / 255
    (x, y, w, h) = coordinates
    w_rm = int(0.2 * w / 2)
    img_rec = img_gray[y:y + h, x + w_rm:x + w - w_rm]
    cut_face = resize(img_rec)
    return cut_face

In [ ]:
def resize(img, size=(64, 64),
           size_percent=None):
    if size_percent is not None:
        return cv2.resize(img, (0, 0), None, size_percent, size_percent)
    if img.shape < size:
        return cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    else:
        return cv2.resize(img, size, interpolation=cv2.INTER_CUBIC)

In [ ]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [ ]:
def adjust_gamma(image, gamma=1.2):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
                      for i in np.arange(0, 256)]).astype("uint8")

    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [ ]:
def alpha_adjust(image):
    alpha = 1.0  # Contrast control (1.0-3.0)
    beta = 0  # Brightness control (0-100)

    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

In [ ]:
def detect_faces(image):
    img = increase_brightness(image)
    gamma_corrected = adjust_gamma(img)
    alpha_corrected = alpha_adjust(gamma_corrected)
    gray_img = cv2.cvtColor(alpha_corrected, cv2.COLOR_BGR2GRAY)
    face_coordinates = trained_face_data.detectMultiScale(gray_img)

    for coordinate in face_coordinates:
        (x, y, w, h) = coordinate
        cut_face = cut_face_from_image(alpha_corrected, coordinate)
        face = gray_img[y:y + h, x:x + w]
        # eye_coordinates = trained_eye_data.detectMultiScale(face)
        colors = np.random.randint(1, 255, 3)

        # cv2.rectangle(image, (x, y), (x + w, y + h), (int(colors[0]), int(colors[1]), int(colors[2])), thickness=2)
        # for (x2, y2, w2, h2) in eye_coordinates:
        #     eye_center = (x + x2 + w2 // 2, y + y2 + h2 // 2)
        #     eye_radius = int(round((w2 + h2) * 0.25))
        #     cv2.circle(image, center=eye_center, radius=eye_radius,
        #                color=(255,0,0))
        norm_image = cv2.normalize(cut_face, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        norm_image = norm_image.astype(np.uint8)
        # norm_image = np.rint((cut_face + 1) * 255 / 2)
        cv2.imshow('cutted_face', norm_image)
        cv2.waitKey(0)
        print(
            f"C:\\Users\\girij\\PycharmProjects\\other resources\\Article\\Fifteenth\\dataset\\img\\img\\image1_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.png")
        cv2.imwrite(
            f"C:\\Users\\girij\\PycharmProjects\\other resources\\Article\\Fifteenth\\dataset\\img\\img\\image1_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.png",
            norm_image)
        # cv2.imshow('Image', cut_face)

    # cv2.waitKey(0)

In [ ]:
image_path = "C:\\Users\\girij\\PycharmProjects\\downloaded_images\\images\\celebrities\\original\\"
image_path = "C:\\Users\\girij\\PycharmProjects\\other resources\\Article\\Fifteenth\\"
image_filename = "musk sahab.jpg"
image_filename = "copy of original.jpg"
webcam = False

In [ ]:
if webcam:
    webcam = cv2.VideoCapture(0)
    while True:
        successful_frame_read, frame = webcam.read()
        detect_faces(frame)
        key = cv2.waitKey(1)
        if key == 81 or key == 113:
            break
    webcam.release()
else:
    # image_list = os.listdir(image_path)
    # for image_filename in image_list:
    img = cv2.imread(image_path + image_filename)
    height = 224
    width = img.shape[1] * height / img.shape[0]
    # img = cv2.resize(img, (int(width), height), None, 0.5, 0.5, interpolation=cv2.INTER_AREA)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    detect_faces(img)

    image = cv2.imread(
        "C:\\Users\\girij\\PycharmProjects\\other resources\\Article\\Fifteenth\\dataset\\img\\img\\image1_20210117090448.png")
    cv2.imshow("image", image)
    cv2.waitKey(0)
